In [ ]:
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch
import torch
from diffusers import StableDiffusionPipeline, DDIMScheduler
from attentionControl import AttentionControlEdit
import diff_latent_attack
from PIL import Image
import numpy as np
import os
import glob
from other_attacks import model_transfer
import random
import sys
from natsort import ns, natsorted
import argparse

In [ ]:
repo_id = "stabilityai/stable-diffusion-2-base"
pipe = DiffusionPipeline.from_pretrained(repo_id, torch_dtype=torch.float16, revision="fp16")

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

prompt = "Best quality, masterpiece, ultra high res, (photorealistic:1.4), raw photo, 1girl, offshoulder, in the dark, deep shadow, low key, cold light"
image = pipe(prompt, num_inference_steps=50).images[0]
image.save("astronaut.png")

In [ ]:

pretrained_diffusion_path = "stabilityai/stable-diffusion-2-base"
ldm_stable = StableDiffusionPipeline.from_pretrained(pretrained_diffusion_path).to('cuda:0')
ldm_stable.scheduler = DDIMScheduler.from_config(ldm_stable.scheduler.config)